In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\jacob\\Desktop\\GDP_Prediction_ML_project'

In [2]:
import pandas as pd

data =pd.read_csv(r'C:\Users\jacob\Desktop\GDP_Prediction_ML_project\artifacts\data_ingestion\quarterly_GDP.csv')
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year and Quarter  278 non-null    object
 1   GDP in Millions   278 non-null    int64 
 2   Date              278 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


In [3]:
data.isnull().sum()

Year and Quarter    0
GDP in Millions     0
Date                0
dtype: int64

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema : dict

In [5]:
from GDP_Prediction.constants import *
from GDP_Prediction.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config


In [7]:
import os
from GDP_Prediction import logger

In [8]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [9]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-11-25 21:19:19,303: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-25 21:19:19,307: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-25 21:19:19,309: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-25 21:19:19,311: INFO: common: created directory at: artifacts]
[2024-11-25 21:19:19,312: INFO: common: created directory at: artifacts/data_validation]
